In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import seaborn as sns
import srsly
from datasets import load_from_disk
from scipy.special import entr, softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from src.data import ClassificationActiveDataModule, ClassificationDataModule
from src.estimators import (
    EstimatorForSequenceClassification,
    RandomStrategyForSequenceClassification,
    # UncertaintyBasedStrategyForSequenceClassification,
    # SEALSRandomStrategyForSequenceClassification,
)

pd.set_option("display.max_colwidth", None)

In [3]:
data_path = Path("../data/prepared/agnews_bert_tiny/")

In [4]:
metadata = srsly.read_yaml(data_path / "metadata.yaml")
tokenizer = AutoTokenizer.from_pretrained(metadata["name_or_path"])

---
### Train

In [ ]:
dataset_dict = load_from_disk(data_path)
train_val = dataset_dict["train"].train_test_split(0.3)
dataset_dict["train"] = train_val["train"]
dataset_dict["validation"] = train_val["test"]

datamodule = ClassificationDataModule.from_dataset_dict(
    dataset_dict, tokenizer=tokenizer, batch_size=64, eval_batch_size=256
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    metadata["name_or_path"],
    id2label=datamodule.id2label,
    label2id=datamodule.label2id,
    num_labels=len(datamodule.labels),
)

In [ ]:
estimator = EstimatorForSequenceClassification(model, accelerator="cuda")

In [ ]:
out = estimator.fit(
    train_loader=datamodule.train_loader(),
    validation_loader=datamodule.validation_loader(),
    limit_train_batches=10,
    limit_validation_batches=30,
    validation_interval=1,
)

In [ ]:
test_out = estimator.test(
    datamodule.test_loader(),
    limit_batches=11,
    # progress_bar=False,
)

---
### Active train

In [5]:
dataset_dict = load_from_disk(data_path)
datamodule = ClassificationDataModule.from_dataset_dict(
    dataset_dict, tokenizer=tokenizer, batch_size=64, eval_batch_size=256
)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['unique_id', 'labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['unique_id', 'labels', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [6]:
active_datamodule = ClassificationActiveDataModule.from_dataset_dict(
    dataset_dict, tokenizer
)

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(
    metadata["name_or_path"],
    id2label=active_datamodule.id2label,
    label2id=active_datamodule.label2id,
    num_labels=len(active_datamodule.labels),
)

Some weights of the model checkpoint at google/bert_uncased_L-2_H-128_A-2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

In [8]:
active_datamodule.data_statistics

{'train_size': 0,
 'validation_size': 0,
 'test_size': 7600,
 'pool_size': 120000,
 'total_labelled_size': 0}

In [9]:
# active_datamodule.load_index(
#     metadata["hnsw_index_path"],
#     embedding_dim=metadata["embedding_dim"],
# )
# active_datamodule.index

In [10]:
RandomStrategyForSequenceClassification(model=model, seed=42).progress_tracker.epoch_tracker

EpochTracker(max=None, total=0, current=0, progress_bar=None)

In [11]:
active_estimator = RandomStrategyForSequenceClassification(model=model, seed=42)

In [12]:
active_estimator.progress_tracker.epoch_tracker

EpochTracker(max=None, total=0, current=0, progress_bar=None)

In [37]:
active_estimator.active_fit(
    active_datamodule,
    max_rounds=2,
    # max_budget=200,
    limit_train_batches=20,
    limit_validation_batches=22,
    limit_test_batches=21,
    limit_pool_batches=30,
    validation_perc=0.4,
    validation_interval=1,
    query_size=25,
)

Completed rounds:   0%|          | 0/2 [00:00<?, ?it/s]

Completed epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch 267:   0%|          | 0/9 [00:00<?, ?it/s]

Validation:   0%|          | 0/6 [00:00<?, ?it/s]

Test:   0%|          | 0/21 [00:00<?, ?it/s]

{'hparams/test_accuracy': 0.711309552192688,
 'hparams/test_f1_macro': 0.7048815488815308,
 'hparams/test_f1_micro': 0.711309552192688,
 'hparams/test_precision_macro': 0.700232207775116,
 'hparams/test_precision_micro': 0.711309552192688,
 'hparams/test_recall_macro': 0.7161346673965454,
 'hparams/test_recall_micro': 0.711309552192688,
 'hparams/test_accuracy_auc': 0.7209822,
 'hparams/test_f1_macro_auc': 0.7048067,
 'hparams/test_f1_micro_auc': 0.7209822,
 'hparams/test_precision_macro_auc': 0.71064305,
 'hparams/test_precision_micro_auc': 0.7209822,
 'hparams/test_recall_macro_auc': 0.71472865,
 'hparams/test_recall_micro_auc': 0.7209822}

In [26]:
active_estimator.progress_tracker.budget_tracker

BudgetTracker(max=119850, total=200, current=200, progress_bar=None, query_size=25)

In [24]:
active_datamodule.data_statistics

{'train_size': 90,
 'validation_size': 60,
 'test_size': 7600,
 'pool_size': 119850,
 'total_labelled_size': 150}

In [ ]:
active_estimator = UncertaintyBasedStrategyForSequenceClassification(
    model=model, score_fn="entropy"
)

In [ ]:
active_estimator.active_fit(
    active_datamodule,
    max_rounds=2,
    max_budget=36500,
    limit_train_batches=20,
    limit_validation_batches=22,
    limit_test_batches=21,
    limit_pool_batches=30,
    # val_perc=0.4,
    query_size=25,
)

In [ ]:
active_datamodule.data_statistics

In [ ]:
df = (
    dataset_dict["validation"]
    .to_pandas()
    .drop(columns=["input_ids", "attention_mask", "token_type_ids"])
    .assign(labels=lambda df_: df_["labels"].map(datamodule.id2label))
)

In [ ]:
df = pd.concat(
    [
        df,
        pd.DataFrame(
            data=softmax(validation_out["logits"], axis=-1),
            columns=datamodule.id2label.values(),
        ),
    ],
    axis=1,
)

In [ ]:
df["entropy"] = entr(df.iloc[:, -4:].values).sum(-1)
df["pred"] = df.iloc[:, -4:].values.argmax(-1)
df["pred"] = df["pred"].map(datamodule.id2label)

In [ ]:
validation_out["metrics"]

In [ ]:
validation_out["unique_id"]

In [ ]:
df = pd.DataFrame(
    data=validation_out["logits"],
    columns=list(range(validation_out["logits"].shape[1])),
).assign(
    pred=lambda df_: df_.values.argmax(-1),
    unique_id=validation_out["unique_id"],
)

In [ ]:
df = pd.merge(
    df, dataset_dict["validation"].to_pandas(), on="unique_id", how="inner"
)

In [ ]:
df = df.iloc[:, :-3]

In [ ]:
df = df.assign(
    pred=lambda df_: df_["pred"].map(datamodule.id2label),
    labels=lambda df_: df_["labels"].map(datamodule.id2label),
)

In [ ]:
(df["pred"] == df["labels"]).mean()

In [ ]:
df["entropy"] = entr(softmax(df.iloc[:, :4], axis=-1)).sum(-1)

In [ ]:
df.sort_values("entropy", ascending=False)